In [ ]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [ ]:
import pandas as pd
df = pd.read_csv("/content/wineQuality.csv")

In [ ]:
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [ ]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [ ]:
x = df.drop("quality", axis=1)
y = df["quality"]

In [ ]:
#Some preprocessing steps before starting to build the model

#To find the features that are contributn more to the model's result
from sklearn.ensemble import ExtraTreesClassifier
find = ExtraTreesClassifier()
find.fit(x,y)
features = pd.Series(find.feature_importances_, index=x.columns)
features.nlargest(5)

alcohol                 0.149930
total sulfur dioxide    0.102162
sulphates               0.101934
volatile acidity        0.097031
density                 0.086123
dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x,y,
                                                test_size=0.2)

In [ ]:
print(xTrain.shape[1])
print(xTest.shape)

11
(320, 11)


In [ ]:
#Building a model

model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, input_shape=(xTrain.shape[1],)),
      tf.keras.layers.Dense(256),
      tf.keras.layers.Dense(512),
      tf.keras.layers.Dense(256),
      tf.keras.layers.Dense(1)
])

model.compile(
    optimizer = "adam",
    loss = tf.keras.losses.mean_squared_error,
    metrics =["mse"]
)

model.fit(xTrain, yTrain, epochs=100)

Epoch 1/100
40/40 [==============================] - 1s 6ms/step - loss: 454.9015 - mse: 454.9015
Epoch 2/100
39/40 [============================>.] - ETA: 0s - loss: 3.9800 - mse: 3.9800

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(xTest, yTest)

In [ ]:
# Now to see the performance of the model

import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
 
predictions = model.predict(xTest)
res = sn.displot(yTest - predictions[:,0])
plt.show()

# As the plot shows a normal distribution this model is working pretty good 

In [ ]:
# So now with a different dataset
df1 = pd.read_csv("/content/house.csv")

In [ ]:
df1.head(140)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,921049315,20140813T000000,199000.0,3,1.75,1320,17390,1.0,0,0,4,7,1320,0,1956,0,98003,47.3257,-122.296,1550,19265
136,3655000070,20140805T000000,220000.0,4,1.75,2020,7840,1.0,0,0,3,7,1010,1010,1968,0,98003,47.3309,-122.299,1750,8140
137,4027700812,20140529T000000,452000.0,4,2.25,2590,10002,1.0,0,0,4,8,1340,1250,1968,0,98028,47.7689,-122.266,1550,10436
138,3992700335,20140707T000000,382500.0,2,1.00,1190,4440,1.0,0,0,3,6,1190,0,1981,0,98125,47.7135,-122.287,1060,5715


In [ ]:
df1.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [ ]:
# Some preprocessing not to have the accurate because here practicing tensorflow regression
df1 = df1.drop(["id","date","zipcode","yr_built","yr_renovated"], axis=1 )
df1.head(2)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,47.7210,-122.319,1690,7639


In [ ]:
x1 = df1.drop("price", axis=1)
y1= df1["price"]

In [ ]:
from sklearn.model_selection import train_test_split
xTrain1, xTest1, yTrain1, yTest1 = train_test_split(x1,y1,
                                                test_size=0.2)

In [ ]:
print(xTrain1.shape)
print(xTest1.shape)


(17290, 15)
(4323, 15)


In [ ]:
#Building a model to predict house price

model1 = tf.keras.Sequential([
      tf.keras.layers.Dense(128, input_shape=(xTrain1.shape[1],)),
      tf.keras.layers.Dense(256),
      tf.keras.layers.Dense(512),
      tf.keras.layers.Dense(256),
      tf.keras.layers.Dense(1)
])

model1.compile(
    optimizer = "adam",
    loss = tf.keras.losses.mae,
    metrics =["mae"]
)

model1.fit(xTrain1, yTrain1, epochs=1000)

In [ ]:
model1.evaluate(xTest1, yTest1)

In [ ]:
# Now to see the performance of the model

import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
 
predictions = model.predict(xTest1)
res = sn.displot(yTest1 - predictions[:,0])
plt.show()


In [ ]:
modelNew = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation = "relu", input_shape=(xTrain1.shape[1],)),
            tf.keras.layers.Dense(256, activation = "relu"),
            tf.keras.layers.Dense(256, activation = "relu"),
            tf.keras.layers.Dense(1)
])

modelNew.compile(
    optimizer = "adam",
    loss = "mse",
    metrics = "mae"
)

modelNew.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 128)               2048      
                                                                 
 dense_14 (Dense)            (None, 256)               33024     
                                                                 
 dense_15 (Dense)            (None, 256)               65792     
                                                                 
 dense_16 (Dense)            (None, 1)                 257       
                                                                 
Total params: 101,121
Trainable params: 101,121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
modelNew.fit(xTrain1, yTrain1, epochs=1000)

Epoch 1/1000
541/541 [==============================] - 2s 3ms/step - loss: 65228713984.0000 - mae: 167010.6875
Epoch 2/1000
541/541 [==============================] - 2s 3ms/step - loss: 63314296832.0000 - mae: 165110.9375
Epoch 3/1000
541/541 [==============================] - 2s 3ms/step - loss: 62743601152.0000 - mae: 163627.3594
Epoch 4/1000
541/541 [==============================] - 1s 3ms/step - loss: 61959663616.0000 - mae: 162669.5000
Epoch 5/1000
541/541 [==============================] - 1s 3ms/step - loss: 61774761984.0000 - mae: 162849.9375
Epoch 6/1000
541/541 [==============================] - 1s 3ms/step - loss: 60470095872.0000 - mae: 160069.8594
Epoch 7/1000
541/541 [==============================] - 1s 3ms/step - loss: 59275845632.0000 - mae: 159371.3906
Epoch 8/1000
541/541 [==============================] - 1s 3ms/step - loss: 59069677568.0000 - mae: 158669.6875
Epoch 9/1000
541/541 [==============================] - 1s 3ms/step - loss: 58748583936.0000 - mae: 1583

In [ ]:
modelNew.predict([[4.0,
 2.25,
 2590.0,
 10002.0,
 1.0,
 0.0,
 0.0,
 4.0,
 8.0,
 1340.0,
 1250.0,
 47.7689,
 -122.266,
 1550.0,
 10436.0]]) # output 452000 [actual price]

array([[426496.75]], dtype=float32)

In [ ]:
modelNew.evaluate(xTest1, yTest1)

136/136 [==============================] - 0s 1ms/step - loss: 48416768000.0000 - mae: 137039.1250


[48416768000.0, 137039.125]